In [1]:
# import functions from Gender CNN notebook
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob  # Can be useful but we use os



SIZE = 300
BATCH_SIZE = 32
CHANNELS = 1  # Grayscale images
INPUT_SHAPE = (SIZE, SIZE, CHANNELS)
GENDER_FEATURE_SHAPE = (1,)  # gender (0 or 1)
EPOCHS = 50
LEARNING_RATE = 4e-4

base_dir = '/content/drive/MyDrive/MLHD'

# Data Paths
train_csv_path = os.path.join(base_dir, 'Train', 'train_labels.csv')
val_csv_path = os.path.join(base_dir, 'Val', 'val_labels.csv')
test_csv_path = os.path.join(base_dir, 'Test', 'test_labels.csv')

train_image_dir = os.path.join(base_dir, 'Train', 'train_samples_pp')
val_image_dir = os.path.join(base_dir, 'Val', 'val_samples_pp')
test_image_dir = os.path.join(base_dir, 'Test', 'test_samples_pp')

checkpoint_filepath = 'Models/gender_model.keras' # tf package says .keras is more efficient


def load_labels(csv_path):
    df = pd.read_csv(csv_path, index_col='id')

    df = df[['boneage', 'male']].rename(columns={'male': 'gender'})
    df['gender'] = df['gender'].astype(np.float32)
    df['boneage'] = df['boneage'].astype(np.float32)
    return df


def create_dataframe(image_dir, labels_df):
    data = []

    for filename in os.listdir(image_dir):
        file_id = int(filename.split('.')[0])
        if file_id in labels_df.index:
            boneage = labels_df.loc[file_id, 'boneage']
            gender = labels_df.loc[file_id, 'gender']
            full_path = os.path.join(image_dir, filename)
            data.append({'file_path': full_path, 'boneage': boneage, 'gender': gender})

    return pd.DataFrame(data)


def preprocess_image(image, labels):
    """preprocesses the image, passes labels through."""
    image = tf.image.resize(image, [SIZE, SIZE])
    # Ensure correct channel dimension if decode_image didn't set it
    if image.shape[-1] is None:
        image = tf.reshape(image, [SIZE, SIZE, CHANNELS])
    elif image.shape[-1] != CHANNELS:

        print(f"Warning: Image has {image.shape[-1]} channels, expected {CHANNELS}. Converting to grayscale.")
        image = tf.image.rgb_to_grayscale(image)  # example if source is RGB

    image = tf.cast(image, tf.float32) / 255.0
    return image, labels  # labels = (boneage, gender)


def image_label_generator(file_paths, boneage_labels, gender_labels):
    """tuple of (boneage, gender) labels."""
    for path, boneage, gender in zip(file_paths, boneage_labels, gender_labels):
        try:
            img_bytes = tf.io.read_file(path)

            image = tf.io.decode_image(img_bytes, channels=CHANNELS, expand_animations=False)

            # allow dynamic height/width initially
            image.set_shape([None, None, CHANNELS])
            yield image, (boneage, gender)  # Yield image and label tuple
        except tf.errors.InvalidArgumentError as e:
            print(f"Warning: Skipping file {path}. Error decoding image: {e}")
        except Exception as e:
            print(f"Warning: Skipping file {path}. Unexpected error: {e}")


def create_tf_dataset(dataframe, shuffle, repeat_flag, batch_size_local=BATCH_SIZE):

    dataset = tf.data.Dataset.from_generator(
        image_label_generator,
        args=[
            dataframe['file_path'].values,
            dataframe['boneage'].values,
            dataframe['gender'].values],
        output_signature=(
            tf.TensorSpec(shape=(None, None, CHANNELS), dtype=tf.uint8),
            (tf.TensorSpec(shape=(), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.float32))))

    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)    # preprocessing

    # Keras multi-input model:
    # map (image, (boneage, gender)) to ((image, gender), boneage)
    dataset = dataset.map(lambda img, labels: ((img, labels[1]), labels[0]),
                          num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe), reshuffle_each_iteration=True)

    if repeat_flag:
        dataset = dataset.repeat()

    dataset = dataset.batch(batch_size_local)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset


# labels
train_labels_df = load_labels(train_csv_path)
val_labels_df = load_labels(val_csv_path)
test_labels_df = load_labels(test_csv_path)

# df
training_dataframe = create_dataframe(train_image_dir, train_labels_df)
validation_dataframe = create_dataframe(val_image_dir, val_labels_df)
test_dataframe = create_dataframe(test_image_dir, test_labels_df)


# repeat=True for train/validation
train_dataset = create_tf_dataset(training_dataframe, shuffle=True, repeat_flag=True)
validation_dataset = create_tf_dataset(validation_dataframe, shuffle=False, repeat_flag=True)
# repeat=False for final evaluation on test set
test_dataset_eval = create_tf_dataset(test_dataframe, shuffle=False, repeat_flag=False)

def gender_model(input_shape, gender_shape):

    # inputs
    image_input = keras.Input(shape=input_shape, name="image_input")
    gender_input = keras.Input(shape=gender_shape, name="gender_input")

    x = layers.Conv2D(32, (3, 3), padding='same', name='conv1a')(image_input)
    x = layers.BatchNormalization(name='bn1a')(x)
    x = layers.Activation('relu', name='relu1a')(x)
    x = layers.Conv2D(32, (3, 3), padding='same', name='conv1b')(x)
    x = layers.BatchNormalization(name='bn1b')(x)
    x = layers.Activation('relu', name='relu1b')(x)
    x = layers.MaxPooling2D((2, 2), name='pool1')(x)  # 500 -> 250

    x = layers.Conv2D(64, (3, 3), padding='same', name='conv2a')(x)
    x = layers.BatchNormalization(name='bn2a')(x)
    x = layers.Activation('relu', name='relu2a')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', name='conv2b')(x)
    x = layers.BatchNormalization(name='bn2b')(x)
    x = layers.Activation('relu', name='relu2b')(x)
    x = layers.MaxPooling2D((2, 2), name='pool2')(x)  # 250 -> 125

    x = layers.Conv2D(128, (3, 3), padding='same', name='conv3a')(x)
    x = layers.BatchNormalization(name='bn3a')(x)
    x = layers.Activation('relu', name='relu3a')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name='conv3b')(x)
    x = layers.BatchNormalization(name='bn3b')(x)
    x = layers.Activation('relu', name='relu3b')(x)
    x = layers.MaxPooling2D((2, 2), name='pool3')(x)  # 125 -> 62

    x = layers.Conv2D(256, (3, 3), padding='same', name='conv4a')(x)
    x = layers.BatchNormalization(name='bn4a')(x)
    x = layers.Activation('relu', name='relu4a')(x)
    x = layers.Conv2D(256, (3, 3), padding='same', name='conv4b')(x)
    x = layers.BatchNormalization(name='bn4b')(x)
    x = layers.Activation('relu', name='relu4b')(x)
    x = layers.MaxPooling2D((2, 2), name='pool4')(x)  # 62 -> 31

    x = layers.Conv2D(256, (3, 3), padding='same', name='conv5a')(x)
    x = layers.BatchNormalization(name='bn5a')(x)
    x = layers.Activation('relu', name='relu5a')(x)
    x = layers.Conv2D(256, (3, 3), padding='same', name='conv5b')(x)
    x = layers.BatchNormalization(name='bn5b')(x)
    x = layers.Activation('relu', name='relu5b')(x)
    x = layers.MaxPooling2D((2, 2), name='pool5')(x)  # 31 -> 15 (approx)

    # feature extraction
    image_features = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)  # (None, 256)

    # fusion with gender
    concatenated_features = layers.concatenate([image_features, gender_input],
                                               name='concatenate_features')  # (None, 257)

    # regression head
    x = layers.Dense(128, name='dense_head1')(concatenated_features)
    x = layers.BatchNormalization(name='bn_head1')(x)
    x = layers.Activation('relu', name='relu_head1')(x)
    x = layers.Dropout(0.4, name='dropout_head')(x)  # Regularization

    bone_age_output = layers.Dense(1, activation='linear', name='bone_age_output')(x)

    model = keras.Model(
        inputs=[image_input, gender_input],
        outputs=bone_age_output,
        name="bone_age_predictor")

    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])  # months

    return model

model = gender_model(INPUT_SHAPE, GENDER_FEATURE_SHAPE)


model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_mae', # save the best mae
    mode='min',
    save_best_only=True)

early_stopping_callback = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,
    mode='min',
    restore_best_weights=True)
# restore best weights because it tends to overfit
# monitor loss because it's the actual improvement metric meanwhile mae can be a face value metric

reduce_lr_callback = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    mode='min',
    min_lr=1e-6)
# trial and error came to a best hyperparam of 3 epochs

callback_list = [model_checkpoint_callback, early_stopping_callback, reduce_lr_callback]

In [2]:
import numpy as np
from sklearn.model_selection import KFold


# how many folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# to store histories
all_hist = []

In [3]:

for fold, (train_idx, val_idx) in enumerate(kf.split(training_dataframe), 1):
    print(f"\n>>> Fold {fold}/{n_splits}")

    # split df
    df_train = training_dataframe.iloc[train_idx]
    df_val   = training_dataframe.iloc[val_idx]

    # build datasets
    train_ds = create_tf_dataset(df_train, shuffle=True, repeat_flag=True)
    val_ds   = create_tf_dataset(df_val,   shuffle=False, repeat_flag=False)

    steps     = len(df_train) // BATCH_SIZE
    val_steps = len(df_val)   // BATCH_SIZE

    # fresh model
    model = gender_model(INPUT_SHAPE, GENDER_FEATURE_SHAPE)

    # fit
    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        steps_per_epoch=steps,
        validation_data=val_ds,
        validation_steps=val_steps,
        callbacks=callback_list,
        verbose=1)

    # save history
    all_hist.append(history.history)


>>> Fold 1/5
Epoch 1/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 134s 206ms/step - loss: 16599.4043 - mae: 122.3636 - val_loss: 16351.3467 - val_mae: 121.1388 - learning_rate: 4.0000e-04
Epoch 2/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 61s 204ms/step - loss: 13995.7500 - mae: 112.0503 - val_loss: 12306.5312 - val_mae: 104.8207 - learning_rate: 4.0000e-04
Epoch 3/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 61s 202ms/step - loss: 10721.3564 - mae: 97.8906 - val_loss: 3015.4346 - val_mae: 46.8515 - learning_rate: 4.0000e-04
Epoch 4/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 60s 200ms/step - loss: 7245.0088 - mae: 79.5228 - val_loss: 5654.0508 - val_mae: 65.4470 - learning_rate: 4.0000e-04
Epoch 5/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 59s 197ms/step - loss: 3912.8035 - mae: 56.8100 - val_loss: 2396.4583 - val_mae: 44.8853 - learning_rate: 4.0000e-04
Epoch 6/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 57s 191ms/step - loss: 1993.7014 - mae: 38.7887 - val_loss: 1462.3503 - val_mae: 30.9677 - learning_rate: 4.0000e-04
Epoch 7/50
301/301 ━━━━━━━━━━━━━━━

In [4]:

# after CV, average your metric, e.g.:
val_maes = np.array([h["val_mae"][-1] for h in all_hist])
print("Per-fold final val MAE:", val_maes)
print("CV mean val MAE:    ", val_maes.mean())
print("CV std  val MAE:    ", val_maes.std())

Per-fold final val MAE: [8.78077793 9.13535118 9.14676666 8.67840099 9.35212135]
CV mean val MAE:     9.018683624267577
CV std  val MAE:     0.2504350974781421
